In [3]:
import pandas as pd
from rpy2.robjects import r, pandas2ri
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification

from sklearn.model_selection import train_test_split
import random
import os as os
import json as json

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import model_selection

import time as time

from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

np.random.seed(1)

In [3]:
#IMPORTING DATA
pandas2ri.activate()
robj = r.load('project/Data/armdata.RData')
data = np.array(r['armdata'])

In [4]:
filename = "armarray" + ".json"
data_save = { "data" : data.tolist() }
with open(filename, 'w') as outfile:
    json.dump(data_save, outfile, indent=4)
print(f"\nStored to file: {filename}")


Stored to file: armarray.json


In [5]:
print(np.shape(data))
#First value is experiment, second is person, third is repetition, fourth is index, and fifth is coordinate, in the format (x,y,z)

(16, 10, 10, 100, 3)


# Section 1 - Predicting object size

In [6]:
#EXTRACTION OF X AND Y FOR MACHINE LEARNING MODEL
Max_Coord = []
Y_experiment = []

cur_x = 0
for experiment in range(15):
    for person in range(10):
        for repetition in range(10):
            cur_z = -99999
            for index in range(100):
                val = data[experiment,person,repetition,index,2]
                if val > cur_z:
                    cur_z = val
                    cur_x = data[experiment,person,repetition,index,0]
            for index in range(100):
                val = data[experiment,person,repetition,index,2]
                if val > cur_z:
                    cur_z = val
                    cur_x = data[experiment,person,repetition,index,0]
            Max_Coord.append([cur_z,cur_x])
            Y_experiment.append(experiment)

In [7]:
#TRANSFORMING PREDICTIONS TO FIT WITH SIZE
Y_experiment = np.array(Y_experiment)
Y_experiment[Y_experiment == 0] = 20
Y_experiment[Y_experiment == 1] = 40
Y_experiment[Y_experiment == 2] = 60
Y_experiment[Y_experiment == 3] = 20
Y_experiment[Y_experiment == 4] = 40
Y_experiment[Y_experiment == 5] = 60
Y_experiment[Y_experiment == 6] = 20
Y_experiment[Y_experiment == 7] = 40
Y_experiment[Y_experiment == 8] = 60
Y_experiment[Y_experiment == 9] = 20
Y_experiment[Y_experiment == 10] = 40
Y_experiment[Y_experiment == 11] = 60
Y_experiment[Y_experiment == 12] = 20
Y_experiment[Y_experiment == 13] = 40
Y_experiment[Y_experiment == 14] = 60


In [8]:
#Making sure lengths are equal as they should be
print(len(Y_experiment))
print(len(Max_Coord))

1500
1500


In [9]:
X_train, X_test, y_train, y_test = train_test_split(Max_Coord, Y_experiment, stratify=Y_experiment,random_state=1)


In [10]:
#NEURAL NETWORK
for i in range(1,10):
    clf = MLPClassifier(random_state=1, max_iter=1000,hidden_layer_sizes=(i*10,i*10)).fit(X_train, y_train)
    print(i)
    print(clf.score(X_test,y_test))


1
0.7226666666666667
2
0.7066666666666667
3
0.7413333333333333
4
0.712
5
0.7653333333333333
6
0.6666666666666666
7
0.5946666666666667
8
0.6693333333333333
9
0.5066666666666667


In [11]:
clf = MLPClassifier(random_state=1, max_iter=1000,hidden_layer_sizes=(50,50)).fit(X_train, y_train)
clf.score(X_test,y_test)

0.7653333333333333

# Section 2 - Predicting object position

In [12]:
#Extracting X and y
Max_Coords_2 = []
Y_experiment_2 = []

cur_x = 0
for experiment in range(15):
    for person in range(10):
        for repetition in range(10):
            cur_z = -99999
            for index in range(100):
                val = data[experiment,person,repetition,index,2]
                if val > cur_z:
                    cur_z = val
                    cur_x = data[experiment,person,repetition,index,0]
            Max_Coords_2.append([cur_z,cur_x])
            Y_experiment_2.append(experiment)

In [13]:
#TRANSFORMING PREDICTIONS TO FIT WITH SIZE
Y_experiment_2 = np.array(Y_experiment_2)
Y_experiment_2[Y_experiment_2 == 0] = 20
Y_experiment_2[Y_experiment_2 == 1] = 20
Y_experiment_2[Y_experiment_2 == 2] = 20
Y_experiment_2[Y_experiment_2 == 3] = 40
Y_experiment_2[Y_experiment_2 == 4] = 40
Y_experiment_2[Y_experiment_2 == 5] = 40
Y_experiment_2[Y_experiment_2 == 6] = 60
Y_experiment_2[Y_experiment_2 == 7] = 60
Y_experiment_2[Y_experiment_2 == 8] = 60
Y_experiment_2[Y_experiment_2 == 9] = 80
Y_experiment_2[Y_experiment_2 == 10] = 80
Y_experiment_2[Y_experiment_2 == 11] = 80
Y_experiment_2[Y_experiment_2 == 12] = 100
Y_experiment_2[Y_experiment_2 == 13] = 100
Y_experiment_2[Y_experiment_2 == 14] = 100

In [14]:
X_train, X_test, y_train, y_test = train_test_split(Max_Coords_2, Y_experiment_2, stratify=Y_experiment_2,random_state=1)


In [15]:
#clt = MLPClassifier(solver='lbfgs', alpha=1e-5,
#    hidden_layer_sizes=(5, 2), random_state=1)
for i in range(1,10):
    clt = MLPClassifier(random_state=1, max_iter=1000,hidden_layer_sizes=(i*10,i*10,i*10,i*10)).fit(X_train, y_train)
    print(clt.score(X_test,y_test))

0.5253333333333333
0.5386666666666666
0.5146666666666667
0.5146666666666667
0.5173333333333333
0.5386666666666666
0.3893333333333333
0.52
0.5333333333333333


In [16]:
clt = MLPClassifier(random_state=1, max_iter=1000,hidden_layer_sizes=(20,20,20,20)).fit(X_train, y_train)
clt.score(X_test, y_test)

0.5386666666666666

# Section 3 - Predicting experiment number

In [17]:
#Importing data
pandas2ri.activate()
robj = r.load('project/Data/armdata.RData')
data = np.array(r['armdata'])


In [18]:
#Extrating X and y
Max_Coord_3 = []
Y_experiment_3 = []

cur_x = 0
for experiment in range(16):
    for person in range(10):
        for repetition in range(10):
            cur_z = -99999
            for index in range(100):
                val = data[experiment,person,repetition,index,2]
                if val > cur_z:
                    cur_z = val
                    cur_x = data[experiment,person,repetition,index,0]
            Max_Coord_3.append([cur_z,cur_x])
            Y_experiment_3.append(experiment)

In [19]:
#Fitting the model
X_train, X_test, y_train, y_test = train_test_split(Max_Coord_3, Y_experiment_3, stratify=Y_experiment_3,random_state=1)
cln = MLPClassifier(random_state=1, max_iter=1000,hidden_layer_sizes=(10,10,10)).fit(X_train, y_train)

In [20]:
#test
cln.score(X_test, y_test)

0.5125

# Section 4 - X Y and Z

In [21]:
#Extrating X and z and y
Max_Coord_4 = []
Z_experiment_4 = []

cur_x = 0
for experiment in range(15):
    for person in range(10):
        for repetition in range(10):
            cur_z = -99999
            cur_y = -99999
            for index in range(100):
                val_z = data[experiment,person,repetition,index,2]
                val_y = data[experiment,person,repetition,index,1]
                if val_z > cur_z:
                    cur_z = val_z
                    cur_x = data[experiment,person,repetition,index,0]
                if abs(val_y) > cur_y:
                    cur_y = val_y
            Max_Coord_4.append([cur_z,cur_x,cur_y])
            Z_experiment_4.append(experiment)


In [22]:
#Fitting the model
X_train, X_test, y_train, y_test = train_test_split(Max_Coord_4, Z_experiment_4, stratify=Z_experiment_4,random_state=1)
cln = MLPClassifier(random_state=1, max_iter=1000,hidden_layer_sizes=(10,10,10)).fit(X_train, y_train)

In [23]:
#test
cln.score(X_test, y_test)

0.44266666666666665

# Alexander Code - Combination with my code

In [4]:

#Importing data
pandas2ri.activate()
robj = r.load('project/Data/armdata.RData')
data = np.array(r['armdata'])

In [5]:
@ignore_warnings(category=ConvergenceWarning)
def train_test(X_train,Y_train,X_test,Y_test, numb = 1, print_B = False):
    #Y is nestes 0-index is heigh, 1 index is position

    logreg3 = LogisticRegression(C=1e5,max_iter=mx_iter)
    logreg3.fit(X_train,Y_train[0] )
    
    logreg3_d = LogisticRegression(C=1e5,max_iter=mx_iter)
    logreg3_d.fit(X_train,Y_train[1] )
    
    y_pred3 = logreg3.predict(X_test)
    y_pred3_d = logreg3_d.predict(X_test)
    
    acc_obj = y_pred3 == Y_test[0]
    acc_dist = y_pred3_d == Y_test[1]
    acc_all = np.logical_and( acc_obj, acc_dist)
    
    acc3_h = np.round(np.mean(acc_obj) ,4)
    acc3_d = np.round(np.mean(acc_dist) ,4)
    
    test_err = np.round(np.mean(acc_all) ,4)
    
    not_N_test = Y_test[0] != "N"

    distances = Y_test[1][not_N_test].astype(float)
    
    y_pred3_d[y_pred3_d == "N"] = "30"
    dist_pred = y_pred3_d[not_N_test].astype(float)
    MSSE = np.mean( (dist_pred - distances)**2 ) 
    
    
    if print_B:
        print(f"\nPREDICTOR{numb}\n","-"*16)
        print(f"\n H: {acc3_h} \n D: {acc3_d}  \n Tot: {test_err}  \n MSSE dist: {MSSE}")
    else:
        return test_err, acc_all
    
    
#TURN ON AND OF TO CHECK THE CONVERGENCE 

In [6]:
@ignore_warnings(category=ConvergenceWarning)
def train_test_reg(X_train,Y_train,X_test,Y_test, numb = 1, print_B = False):
    #Y is nestes 0-index is heigh, 1 index is position

    logreg3 = LogisticRegression(C=1e5,max_iter=mx_iter)
    logreg3.fit(X_train,Y_train[0] )
    
    not_N = Y_train[0] != "N"
    
    linreg3_d = LinearRegression()
    linreg3_d.fit(X_train[not_N],Y_train[1][not_N] )
    
    y_pred3 = logreg3.predict(X_test)
    y_pred3_d = linreg3_d.predict(X_test)
    
    y_pred_d_String = np.empty((len(y_pred3_d)), dtype=object)
    
    y_pred_d_String[y_pred3_d/7.5 < 2.5] = "15"
    y_pred_d_String[ np.logical_and( y_pred3_d/7.5 > 2.5 , y_pred3_d/7.5 < 3.5 )] = "22.5"
    y_pred_d_String[ np.logical_and( y_pred3_d/7.5 > 3.5 , y_pred3_d/7.5 < 4.5 )] = "30"
    y_pred_d_String[ np.logical_and( y_pred3_d/7.5 > 4.5 , y_pred3_d/7.5 < 5.5 )] = "37.5"
    y_pred_d_String[ y_pred3_d/7.5 > 5.5] = "45"
    
    y_pred_d_String[y_pred3 == "N"] = "N"
    
    acc_obj = y_pred3 == Y_test[0]
    acc_dist = y_pred_d_String == Y_test[1]
    acc_all = np.logical_and( acc_obj, acc_dist)
    
    acc3_h = np.round(np.mean(acc_obj) ,4)
    acc3_d = np.round(np.mean(acc_dist) ,4)
    
    test_err = np.round(np.mean(acc_all) ,4)
    
    not_N_test = Y_test[0] != "N"

    distances = Y_test[1][not_N_test].astype(float)
    dist_pred = y_pred3_d[not_N_test].astype(float)
    
    MSSE = np.mean( (dist_pred - distances)**2 ) 
    
    """
    i_s = np.random.randint(len(Y_d),size =36)
    for i in i_s:
        print(y_pred3_d[i],Y_d[i],y_pred_d_String[i])
        """
    if print_B:
        print(f"\nPREDICTOR(reg){numb}\n","-"*16)
        print(f"\n H: {acc3_h} \n D: {acc3_d}  \n Tot: {test_err} \n MSSE dist: {MSSE}")
    else:
        return test_err, acc_all

In [7]:
tot = 16*10*10
X = np.empty((tot,16))
X_raw = np.empty((tot,300))
y = np.empty((tot), dtype=object)
y_d = np.empty((tot), dtype=object)

y_ex = np.empty((tot), dtype=object)
y_per = np.empty((tot), dtype=object)
y_tr = np.empty((tot), dtype=object)

remove_indx = []

num_for_ang = 10

for i in range(tot):
    ex = i//100
    per = (i//10)%10
    tr = i%10
    
    y_ex[i] = ex
    y_per[i] = per
    y_tr[i] = tr
    
    if per == 8:
        if [ex,tr] in [[4,0],[6,1],[9,0],[10,0],[12,0],[13,1]]:
            remove_indx.append(i)
            
            continue
    
    cords = data[ex,per,tr]
    
    #Max heigh
    indx_z = np.argmax(cords,axis = 0)[2]
    max_z = np.array([cords[indx_z,2],cords[indx_z,0],indx_z])
    
    #Max transverse
    indx_y = np.argmax(abs(cords),axis = 0)[1]
    max_y = np.array([abs(cords[indx_y,1]),cords[indx_y,0],indx_y])
    
    # 6 for this part
    
    #Angle-z
    
    regressor = LinearRegression()
    regressor.fit(cords[:num_for_ang,0].reshape(-1, 1), cords[:num_for_ang,2].reshape(-1, 1))
    ang_z = np.sin(regressor.coef_[0][0])
    
    regressor = LinearRegression()
    regressor.fit(cords[-num_for_ang:,0].reshape(-1, 1), cords[-num_for_ang:,2].reshape(-1, 1))
    ang_z2 = np.sin(regressor.coef_[0][0])
    
    #Time
    regressor = LinearRegression()
    regressor.fit(np.arange(num_for_ang).reshape(-1, 1), cords[:num_for_ang,2].reshape(-1, 1))
    ang_z_t = np.sin(regressor.coef_[0][0])
    
    regressor = LinearRegression()
    regressor.fit(np.arange(num_for_ang).reshape(-1, 1), cords[-num_for_ang:,2].reshape(-1, 1))
    ang_z2_t = np.sin(regressor.coef_[0][0])
    

    
    #Angle-y
    
    regressor = LinearRegression()
    regressor.fit(cords[:num_for_ang,0].reshape(-1, 1), cords[:num_for_ang,1].reshape(-1, 1))
    ang_y = np.sin(regressor.coef_[0][0])
    
    regressor = LinearRegression()
    regressor.fit(cords[-num_for_ang:,0].reshape(-1, 1), cords[-num_for_ang:,1].reshape(-1, 1))
    ang_y2 = np.sin(regressor.coef_[0][0])
    
    #Time
    regressor = LinearRegression()
    regressor.fit(np.arange(num_for_ang).reshape(-1, 1), cords[:num_for_ang,2].reshape(-1, 1))
    ang_y_t = np.sin(regressor.coef_[0][0])
    
    regressor = LinearRegression()
    regressor.fit(np.arange(num_for_ang).reshape(-1, 1), cords[-num_for_ang:,2].reshape(-1, 1))
    ang_y2_t = np.sin(regressor.coef_[0][0])
    
    angles = np.array([ang_z,ang_z2,ang_z_t,ang_z2_t,
                       ang_y,ang_y2,ang_y_t,ang_y2_t])
    
    
    X[i,0:14] = np.concatenate((max_z,max_y,angles))
    X_raw[i] = cords.flatten('F')

In [8]:
#indexeses for maxes
inxs_noty_mx = np.array([0,1,2]) #max height (z)

inxs_notz_mx = np.array([3,4,5]) #max transverse (y)

inxs_all_mx = np.array([0,1,2,3,4,5]) # Both z and y

inxs_notx_mx = np.array([0,2,3,5]) # Excluding x info

In [9]:
#indexeses for angles
inxs_noty_ang = np.array([6,7,8,9]) # (z)

inxs_notz_ang = np.array([10,11,12,13]) # (y)

inxs_all_ang = np.array([6,7,8,9,10,11,12,13]) # Both z and y

inxs_notx_ang = np.array([8,9,12,13]) # Excluding x info

In [10]:
#Final indexes

inxs_all = np.concatenate((inxs_all_mx,inxs_all_ang))

inxs_noty = np.concatenate((inxs_noty_mx,inxs_noty_ang))

inxs_notz = np.concatenate((inxs_notz_mx,inxs_notz_ang))

inxs_notx = np.concatenate((inxs_notx_mx,inxs_notx_ang))

In [11]:
y_ob_h = np.empty((tot), dtype=object)
y_ob_d = np.empty((tot), dtype=object)

i_arr = np.arange(tot)
y_ob_h[y_ex%3 == 0] = "S"
y_ob_h[y_ex%3 == 1] = "M"
y_ob_h[y_ex%3 == 2] = "T"
y_ob_h[y_ex == 15] = "N"

y_ob_d[y_ex//3 == 0] = "15"
y_ob_d[y_ex//3 == 1] = "22.5"
y_ob_d[y_ex//3 == 2] = "30"
y_ob_d[y_ex//3 == 3] = "37.5"
y_ob_d[y_ex//3 == 4] = "45"
y_ob_d[y_ex//3 == 5] = "N"

In [12]:
mx_iter = int(1e20)

X = np.delete(X,remove_indx,axis=0)
Y = np.delete(y_ob_h,remove_indx)
Y_d = np.delete(y_ob_d,remove_indx)

X_raw = np.delete(X_raw,remove_indx,axis=0)
y_ex = np.delete(y_ex, remove_indx)

raw_inx_all = np.arange(300)
raw_inx_noty = np.concatenate((np.arange(100),np.arange(200,300)))
raw_inx_notz = np.arange(200)
raw_inx_notx = np.arange(100,300)



K_num = 5

CV = model_selection.KFold(K_num, shuffle=True)
folds = [[i,j] for i,j in CV.split(X,Y)]

y_per = np.delete(y_per,remove_indx)
group_kfold = model_selection.GroupKFold(n_splits=5)
folds_B = [[i,j] for i,j in group_kfold.split(X, Y, y_per)]

In [15]:

train_indx, test_indx = folds_B[0][0],folds_B[0][1]

X_train = X[train_indx,:]
X_test = X[test_indx,:]

Y_train = [Y[train_indx], Y_d[train_indx], y_ex[train_indx]]
Y_test = [Y[test_indx], Y_d[test_indx], y_ex[test_indx]]

X_train_raw = X_raw[train_indx,:]
X_test_raw = X_raw[test_indx,:]


In [16]:
def train_test_neural_network(X_train_vals, Y_train_vals, X_test_vals, Y_test_vals, numb=1, print_B = False):
    sizes_train = []
    for i in Y_train_vals[0]:
        if i == "S":
            sizes_train.append(1)
        elif i == "M":
            sizes_train.append(2)
        elif i == "T":
            sizes_train.append(3)
        elif i == "N":
            sizes_train.append(4)
        else:
            print("ERROR: NOT EQUAL S M OR T")
    
    sizes_test = []
    for i in Y_test_vals[0]:
        if i == "S":
            sizes_test.append(1)
        elif i == "M":
            sizes_test.append(2)
        elif i == "T":
            sizes_test.append(3)
        elif i == "N":
            sizes_test.append(4)
        else:
            print("ERROR: NOT EQUAL S M OR T")

    distances_train = []
    for i in Y_train_vals[1]:
        if i == "N":
            distances_train.append(100)
        else:
            distances_train.append(i)
    
    distances_test = []
    for i in Y_test_vals[1]:
        if i == "N":
            distances_test.append(100)
        else:
            distances_test.append(i)
    clsize = MLPClassifier(random_state=1, max_iter=1000,hidden_layer_sizes=(20,20,20)).fit(X_train_vals, sizes_train)
    
    cldist = MLPClassifier(random_state=1, max_iter=1000,hidden_layer_sizes=(20,20,20)).fit(X_train_vals, distances_train)
    
    clexer = MLPClassifier(random_state=1, max_iter=1000,hidden_layer_sizes=(20,20,20)).fit(X_train_vals, list(Y_train_vals[2])) 
    if print_B:
        print(f"\nPREDICTOR LINEAR REGRESSION{numb}\n","-"*16)
        print(" ")
        print("H = " + str(clsize.score(X_test_vals,sizes_test)))
        
        print("D = " + str(cldist.score(X_test_vals,distances_test)))

        print("tot = " + str(clexer.score(X_test_vals, list(Y_test_vals[2]))))
        print(" ")
    else:
        return clsize.score(X_test_vals,sizes_test), cldist.score(X_test_vals,distances_test), clexer.score(X_test_vals, list(Y_test_vals[2]))


In [17]:
test_list = [inxs_all,inxs_all_mx,inxs_noty,inxs_notz,inxs_notx]
for i in range(len(test_list)):
    
    indx_curr = test_list[i]
    
    acc_1, vals_1 = train_test(X_train[:,indx_curr], Y_train, X_test[:,indx_curr], Y_test, numb = "_"+str(i),print_B = False)
    
    
    acc_2, vals_2 = train_test_reg(X_train[:,indx_curr], Y_train, X_test[:,indx_curr], Y_test, numb = "_"+str(i),print_B = False)
    
    acc_size, acc_dist, acc_score = train_test_neural_network(X_train[:,indx_curr], Y_train, X_test[:,indx_curr], Y_test, numb = "_"+str(i),print_B = False)
    
    print(str(acc_1) + ",")
    print(str(acc_2) + ",")
    print(str(acc_score) + ",")
    
    



0.3312,
0.3188,
0.328125,
0.3719,
0.3594,
0.33125,
0.3875,
0.3812,
0.4,
0.1375,
0.1375,
0.18125,
0.2156,
0.175,
0.253125,


In [59]:
    
tests_raw = [raw_inx_all,raw_inx_noty,raw_inx_notz,raw_inx_notx]

for i in range(len(tests_raw)):
    
    indx_curr = tests_raw[i]
    
    train_test(X_train_raw[:,indx_curr], Y_train, X_test_raw[:,indx_curr], Y_test, numb = "RAW_"+str(i),print_B = True)
    
    train_test_reg(X_train_raw[:,indx_curr], Y_train, X_test_raw[:,indx_curr], Y_test, numb = "RAW_"+str(i),print_B = True)
    
    train_test_neural_network(X_train_raw[:,indx_curr], Y_train, X_test_raw[:,indx_curr], Y_test, numb = "RAW_"+str(i))



PREDICTOR LINEAR REGRESSIONRAW_0
 ----------------
 
H = 0.7
D = 0.43125
tot = 0.4
 

PREDICTOR LINEAR REGRESSIONRAW_1
 ----------------
 
H = 0.778125
D = 0.4625
tot = 0.484375
 

PREDICTOR LINEAR REGRESSIONRAW_2
 ----------------
 
H = 0.46875
D = 0.45625
tot = 0.259375
 

PREDICTOR LINEAR REGRESSIONRAW_3
 ----------------
 
H = 0.740625
D = 0.34375
tot = 0.4125
 


# Cross validation

In [45]:
#FOR ALL VALS : PROCESSED

test_list = [inxs_all,inxs_all_mx,inxs_noty,inxs_notz,inxs_notx]
#for i in range(len(test_list)):
accuracies_logistic = []
accuracies_neural = []
acc_baselines = []


for f in range(5):
    train_indx, test_indx = folds_B[f][0],folds_B[f][1]

    X_train = X[train_indx,:]
    X_test = X[test_indx,:]

    Y_train = [Y[train_indx], Y_d[train_indx], y_ex[train_indx]]
    Y_test = [Y[test_indx], Y_d[test_indx], y_ex[test_indx]]


    indx_curr = test_list[4]
    
    acc_1, vals_1 = train_test(X_train[:,indx_curr], Y_train, X_test[:,indx_curr], Y_test, numb = "_"+str(i),print_B = False)
    
    
    #acc_2, vals_2 = train_test_reg(X_train[:,indx_curr], Y_train, X_test[:,indx_curr], Y_test, numb = "_"+str(i),print_B = False)
    
    acc_size, acc_dist, acc_score = train_test_neural_network(X_train[:,indx_curr], Y_train, X_test[:,indx_curr], Y_test, numb = "_"+str(i),print_B = False)
    
    acc_baselines.append(np.mean(Y_test[2] == 0))
    
    
    accuracies_logistic.append(acc_1)
    accuracies_neural.append(acc_score)

In [50]:
print(acc_baselines)
print(accuracies_logistic)
print(accuracies_neural)

[0.0625, 0.0625, 0.0625, 0.0625, 0.06369426751592357]
[0.2156, 0.2344, 0.3219, 0.2969, 0.2197]
[0.253125, 0.2875, 0.328125, 0.290625, 0.2070063694267516]


In [52]:
#RAW ALL DIMENSION
    
tests_raw = [raw_inx_all,raw_inx_noty,raw_inx_notz,raw_inx_notx]

accuracies_logistic_raw = []
accuracies_neural_raw = []

for f in range(5):
    
    train_indx, test_indx = folds_B[f][0],folds_B[f][1]

    X_train = X[train_indx,:]
    X_test = X[test_indx,:]

    Y_train = [Y[train_indx], Y_d[train_indx], y_ex[train_indx]]
    Y_test = [Y[test_indx], Y_d[test_indx], y_ex[test_indx]]

    X_train_raw = X_raw[train_indx,:]
    X_test_raw = X_raw[test_indx,:]

    indx_curr = tests_raw[3]
    
    acc_3, vals_3 = train_test(X_train_raw[:,indx_curr], Y_train, X_test_raw[:,indx_curr], Y_test, numb = "RAW_"+str(i),print_B = False)
    
    #train_test_reg(X_train_raw[:,indx_curr], Y_train, X_test_raw[:,indx_curr], Y_test, numb = "RAW_"+str(i),print_B = True)
    
    acc_size_4, acc_dist_4, acc_score_4 = train_test_neural_network(X_train_raw[:,indx_curr], Y_train, X_test_raw[:,indx_curr], Y_test, numb = "RAW_"+str(i), print_B = False)
    
    accuracies_logistic_raw.append(acc_3)
    accuracies_neural_raw.append(acc_score_4)
    
    

In [51]:
print(accuracies_logistic_raw)
print(accuracies_neural_raw)

[0.2219, 0.2312, 0.2188, 0.2281, 0.1497]
[0.259375, 0.196875, 0.290625, 0.234375, 0.17197452229299362]


# X left and X Raw left matrices

In [28]:
from copy import deepcopy
X_LEFT = np.empty((tot,16))
X_raw_LEFT = np.empty((tot,300))
for i in range(tot):
    ex = i//100
    per = (i//10)%10
    tr = i%10
    
    if per == 8:
        if [ex,tr] in [[4,0],[6,1],[9,0],[10,0],[12,0],[13,1]]:
            
            continue
    
    cords = data[ex,per,tr]
    
    cords_left = deepcopy(cords)
    cords_left[:,1] = -1*cords_left[:,1]
    
    cords = cords_left
    
    #Max heigh
    indx_z = np.argmax(cords,axis = 0)[2]
    max_z = np.array([cords[indx_z,2],cords[indx_z,0],indx_z])
    
    #Max transverse
    indx_y = np.argmax(abs(cords),axis = 0)[1]
    max_y = np.array([abs(cords[indx_y,1]),cords[indx_y,0],indx_y])
    
    # 6 for this part
    
    #Angle-z
    
    regressor = LinearRegression()
    regressor.fit(cords[:num_for_ang,0].reshape(-1, 1), cords[:num_for_ang,2].reshape(-1, 1))
    ang_z = np.sin(regressor.coef_[0][0])
    
    regressor = LinearRegression()
    regressor.fit(cords[-num_for_ang:,0].reshape(-1, 1), cords[-num_for_ang:,2].reshape(-1, 1))
    ang_z2 = np.sin(regressor.coef_[0][0])
    
    #Time
    regressor = LinearRegression()
    regressor.fit(np.arange(num_for_ang).reshape(-1, 1), cords[:num_for_ang,2].reshape(-1, 1))
    ang_z_t = np.sin(regressor.coef_[0][0])
    
    regressor = LinearRegression()
    regressor.fit(np.arange(num_for_ang).reshape(-1, 1), cords[-num_for_ang:,2].reshape(-1, 1))
    ang_z2_t = np.sin(regressor.coef_[0][0])
    

    
    #Angle-y
    
    regressor = LinearRegression()
    regressor.fit(cords[:num_for_ang,0].reshape(-1, 1), cords[:num_for_ang,1].reshape(-1, 1))
    ang_y = np.sin(regressor.coef_[0][0])
    
    regressor = LinearRegression()
    regressor.fit(cords[-num_for_ang:,0].reshape(-1, 1), cords[-num_for_ang:,1].reshape(-1, 1))
    ang_y2 = np.sin(regressor.coef_[0][0])
    
    #Time
    regressor = LinearRegression()
    regressor.fit(np.arange(num_for_ang).reshape(-1, 1), cords[:num_for_ang,2].reshape(-1, 1))
    ang_y_t = np.sin(regressor.coef_[0][0])
    
    regressor = LinearRegression()
    regressor.fit(np.arange(num_for_ang).reshape(-1, 1), cords[-num_for_ang:,2].reshape(-1, 1))
    ang_y2_t = np.sin(regressor.coef_[0][0])
    
    angles = np.array([ang_z,ang_z2,ang_z_t,ang_z2_t,
                       ang_y,ang_y2,ang_y_t,ang_y2_t])
    
    
    X_LEFT[i,0:14] = np.concatenate((max_z,max_y,angles))
    X_raw_LEFT[i] = cords.flatten('F')
    
X_raw_LEFT = np.delete(X_raw_LEFT,remove_indx,axis=0)
X_LEFT = np.delete(X_LEFT,remove_indx,axis=0)

In [30]:

test_list = [inxs_all,inxs_all_mx,inxs_noty,inxs_notz,inxs_notx]
#for i in range(len(test_list)):
accuracies_logistic_LEFT = []
accuracies_neural_LEFT = []
for f in range(5):
        
    train_indx, test_indx = folds_B[f][0],folds_B[f][1]

    X_train = X[train_indx,:]
    X_test = X_LEFT[test_indx,:]

    Y_train = [Y[train_indx], Y_d[train_indx], y_ex[train_indx]]
    Y_test = [Y[test_indx], Y_d[test_indx], y_ex[test_indx]]


    indx_curr = test_list[0]
    
    acc_1, vals_1 = train_test(X_train[:,indx_curr], Y_train, X_test[:,indx_curr], Y_test, numb = "_"+str(i),print_B = False)
    
    
    #acc_2, vals_2 = train_test_reg(X_train[:,indx_curr], Y_train, X_test[:,indx_curr], Y_test, numb = "_"+str(i),print_B = False)
    
    acc_size, acc_dist, acc_score = train_test_neural_network(X_train[:,indx_curr], Y_train, X_test[:,indx_curr], Y_test, numb = "_"+str(i),print_B = False)
    
    accuracies_logistic_LEFT.append(acc_1)
    accuracies_neural_LEFT.append(acc_score)

In [31]:
print(accuracies_logistic_LEFT)
print(accuracies_neural_LEFT)

[0.3188, 0.3938, 0.4688, 0.4188, 0.2739]
[0.365625, 0.378125, 0.525, 0.359375, 0.2929936305732484]


In [32]:
#RAW ALL DIMENSION
    
tests_raw = [raw_inx_all,raw_inx_noty,raw_inx_notz,raw_inx_notx]

accuracies_logistic_raw_LEFT = []
accuracies_neural_raw_LEFT = []

for f in range(5):
    
    train_indx, test_indx = folds_B[f][0],folds_B[f][1]

    X_train = X[train_indx,:]
    X_test = X[test_indx,:]

    Y_train = [Y[train_indx], Y_d[train_indx], y_ex[train_indx]]
    Y_test = [Y[test_indx], Y_d[test_indx], y_ex[test_indx]]

    X_train_raw = X_raw[train_indx,:]
    X_test_raw = X_raw_LEFT[test_indx,:]

    indx_curr = tests_raw[0]
    
    acc_3, vals_3 = train_test(X_train_raw[:,indx_curr], Y_train, X_test_raw[:,indx_curr], Y_test, numb = "RAW_"+str(i),print_B = False)
    
    #train_test_reg(X_train_raw[:,indx_curr], Y_train, X_test_raw[:,indx_curr], Y_test, numb = "RAW_"+str(i),print_B = True)
    
    acc_size_4, acc_dist_4, acc_score_4 = train_test_neural_network(X_train_raw[:,indx_curr], Y_train, X_test_raw[:,indx_curr], Y_test, numb = "RAW_"+str(i), print_B = False)
    
    accuracies_logistic_raw_LEFT.append(acc_3)
    accuracies_neural_raw_LEFT.append(acc_score_4)

In [33]:
print(accuracies_logistic_raw_LEFT)
print(accuracies_neural_raw_LEFT)

[0.1656, 0.1812, 0.2062, 0.1781, 0.1465]
[0.1625, 0.190625, 0.234375, 0.078125, 0.1624203821656051]
